<a href="https://colab.research.google.com/github/oObelix/DataScience/blob/master/Jun_ML_Classification_2_HW_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 2. Метрики качества классификации ч.1

В уроке мы разбирали матрицу ошибок и некоторые метрики, основанные на терминах матрицы ошибок. Здесь же посмотрим, как эти метрики применять на практике и что  по ним можно выяснить.

### 1.1
Выкачайте тренировочный датасет Titanic - train.csv - с сайта [kaggle](https://www.kaggle.com/c/titanic/data). С помощью функции pd.read_csv() загрузите данные в датафрейм. Выведите первые 20 строк и проанализируйте данные: какие колонки присутствуют (более конкретная информация по ним есть на сайте kaggle), каким образом в них обозначены данные и какие типы данных используются (используйте pandas.dtypes). 

In [ ]:
import pandas as pd

In [ ]:
### YOUR CODE HERE ###

df = pd.read_csv('sample_data/train.csv')

In [ ]:
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [ ]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

### 1.2
Проверьте, имеются ли пропущенные значения в колонках, и выведите сумму всех пропущенных значений в каждой из колонок.

In [ ]:
### YOUR CODE HERE ###

df.isnull().sum(axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 1.3
Замените все пропущенные значения колонки Age на медианы в зависимости от пола человека: т.е. если пол человека в строке с пропущенным значением "male", заменяете пропущенное значение возраста на медиану по всем известным возрастам мужчин, и наоборот. Выведите медианы возраста в зависимости от пола. Пропущенные значения колонок Cabin и Embarked замените на U (Unknown).

Примечание: для группировки по полу можно использовать метод df.groupby(), который имеет встроенные агрегатные функции (в т.ч. для вычисления медианы).  Для установки пропущенных значений в датасете воспользуйтесь функцией df.apply().

In [ ]:
### YOUR CODE HERE ###

age_median = df.groupby('Sex')['Age'].median()
print(age_median['female'], age_median['male'])
# for key, item in man_median:
#     print(man_median.get_group(key), "\n\n")

27.0 29.0


In [ ]:
# с пользовательской функцией без lambda
def age_replace(row, age_median_female, age_median_male):
  if pd.isnull(row['Cabin']):
    row['Cabin'] = 'U'
  if pd.isnull(row['Embarked']):
    row['Embarked'] = 'U'
  if pd.isnull(row['Age']):
    if row['Sex'] == 'female':
      row['Age'] = age_median_female
    else:
      row['Age'] = age_median_male
  return row

df_age_median = df.apply(age_replace, args = [age_median['female'], 
                                              age_median['male']], axis = 1)
df_age_median.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,U,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,U,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,U,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,U,C


### 1.4
Выведите возраст пассажиров с PassengerID = [6, 20]. Убедитесь, что заполнены все пропущенные значения (воспользуйтесь функцией df.isnull() ).

In [ ]:
### YOUR CODE HERE ###

df_age_median[df_age_median['PassengerId'].isin([6, 20])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,U,Q
19,20,1,3,"Masselmani, Mrs. Fatima",female,27.0,0,0,2649,7.2250,U,C


In [ ]:
df_age_median[(df_age_median['PassengerId'] > 6) & (df_age_median['PassengerId'] < 20)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,U,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,U,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,U,C
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,U,S
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,U,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,U,S
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,U,S


Так как не все классификаторы могут работать с категориальными признаками, в данном случае необходимо заменить их на числовые.

Также можно было бы выделить расширенные признаки, такие как статус пассажира (Mr., Mrs., Miss., Dr., Master. и т.д.), собственный индекс билета, индекс палубы и другие, но здесь ограничимся выбором более простых признаков.

In [ ]:
# df_age_median.replace({'Cabin': {'A(.*)': '1\\1', 'B(.*)': '2\\1', 
#                                  'C(.*)': '2\\1', 'E(.*)': '4\\1'}}, regex=True)

### 1.5
В колонке Sex замените значения на 0, если пол "male", и на 1, если "female". В колонке Embarked замените параметры "U", "S", "C", "Q" на 0, 1, 2, 3 соответственно. Отбросьте колонки PassengerId, Name, Ticket, Cabin. Выведите первые 20 строк получившегося набора данных.

In [ ]:
### YOUR CODE HERE ###

df_1 = df_age_median.replace({'Sex': {'male': 0, 'female': 1}, 
                       'Embarked': {"U": 0, "S": 1, "C": 2, "Q": 3}}) \
                       .drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
df_1.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1
5,0,3,0,29.0,0,0,8.4583,3
6,0,1,0,54.0,0,0,51.8625,1
7,0,3,0,2.0,3,1,21.0750,1
8,1,3,1,27.0,0,2,11.1333,1
9,1,2,1,14.0,1,0,30.0708,2


### 1.6
Приступим к построению классификаторов. **Условимся, что, если функция или объект модели имеют параметр random_state, то устанавливаем его равным 17 в каждом из случаев.**

Разделите данные на тренировочный и тестовый датасеты, установив размер тестового как 0.25 (первая колонка Survived является целевой, поэтому необходимо сначала ее отделить от признаков).

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
### YOUR CODE HERE ###

df_survived = df_1['Survived']
# df_survived
df_2 = df_1.drop(columns=['Survived'])
# df_2

x_train, x_test, y_train, y_test = train_test_split(df_2, df_survived, 
                                                    test_size=0.25, 
                                                    random_state=17)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(668, 7) (223, 7) (668,) (223,)


### 1.7
Обучите на полученных выборках несколько классификаторов, которые импортированы ниже.

**Примечание: в методе ближайших соседей используйте количество соседей, равное 5.**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
### YOUR CODE HERE ###

model_knc = KNeighborsClassifier(n_neighbors=5).fit(x_train, y_train)
model_gnb = GaussianNB().fit(x_train, y_train)
model_dtc = DecisionTreeClassifier().fit(x_train, y_train)
model_lrg = LogisticRegression().fit(x_train, y_train)

Очевидно, в этом случае использовать метрику accuracy, которую мы применяли до этого, не совсем правильно: она не даст верную оценку классификатору, потому как выживших явно меньше, чем погибших.

Вспомним про такие метрики как точность и полнота. Здесь они подойдут хорошо, так как у нас имеются положительный и отрицательный классы, и определить корректно один из них может оказаться более важным .

### 1.8
В каждом из пунктов следующего теста выберите, что важнее максимизировать: точность (precision) или полноту (recall). Выпишите ответы.

1. Вероятность того, что при определенной поломке самолета он сможет долететь до пункта назначения (1 - долетел, 0 - не долетел).
2. Предсказание, представляет ли человек опасность, по анализу психического состояния (1 - представляет опасность, 0 - не представляет опасности).
3. Предсказание ухода клиента (1 - клиент ушел, 0 - остался).
4. Выявление рака на основе медицинских показателей (1 - болен раком, 0 - здоров).
5. Предсказание летальности при наблюдаемой мутации (1 - выживание, 0 - летальный исход).
6. Определение важности происшествия для экстренных служб (1 - важно, 0 - неважно).
7. Окупятся ли вложения в бизнес (1 - окупятся, 0 - не окупятся).

Ваш ответ:

1. precision<br>
2. recall - это если мы не боимся ложно обвинить человека нездоровым и опасным!<br>
3. precision<br>
4. recall<br>
5. precision<br>
6. recall<br>
7. precision<br>

### 1.9
Определите, что важнее использовать в случае датасета Titanic: полноту или точность. Объясните, почему.


## Важно знать точность. Мы НЕ должны завышать колличество выживших. Правильнее в катастрофе считать человека погибшим, пока не доказанно обратное.

<br>
<br>




### 1.10
Для каждого классификатора выведите матрицу ошибок и самостоятельно рассчитайте метрику recall (расчеты должны присутствовать). Проверьте расчеты, воспользовавшись встроенной метрикой recall_score.

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score

In [ ]:
### YOUR CODE HERE ###

# 1) Что имеется ввиду под "Для каждого классификатора"?
# 2) Какие данные использовать для проверки их recall расчётами и recall_score?

print('KNeighborsClassifier')
y_pred = model_knc.predict(x_test)
cm_knc = confusion_matrix(y_test, y_pred)
print(cm_knc)
# матрица оказалась вывернута наизнанку относительно той, что мы изучали в лекциях
# её структура имеет вид
# [[TN, FP]
#  [FN, TP]]
# TP = cm_knc[1][1], FN = cm_knc[1][0]
print('recall =       ', cm_knc[1][1] / (cm_knc[1][1] + cm_knc[1][0]))
print('recall_score = ', recall_score(y_test, y_pred))
print('-'*30)

print('GaussianNB')
y_pred = model_gnb.predict(x_test)
cm_gnb = confusion_matrix(y_test, y_pred)
print(cm_gnb)
print('recall =       ', cm_gnb[1][1] / (cm_gnb[1][1] + cm_gnb[1][0]))
print('recall_score = ', recall_score(y_test, y_pred))
print('-'*30)

print('DecisionTreeClassifier')
y_pred = model_dtc.predict(x_test)
cm_dtc = confusion_matrix(y_test, y_pred)
print(cm_dtc)
print('recall =       ', cm_dtc[1][1] / (cm_dtc[1][1] + cm_dtc[1][0]))
print('recall_score = ', recall_score(y_test, y_pred))
print('-'*30)

print('LogisticRegression')
y_pred = model_lrg.predict(x_test)
cm_lrg = confusion_matrix(y_test, y_pred)
print(cm_lrg)
print('recall =       ', cm_lrg[1][1] / (cm_lrg[1][1] + cm_lrg[1][0]))
print('recall_score = ', recall_score(y_test, y_pred))
print('-'*30)

KNeighborsClassifier
[[110  21]
 [ 47  45]]
recall =        0.4891304347826087
recall_score =  0.4891304347826087
------------------------------
GaussianNB
[[111  20]
 [ 27  65]]
recall =        0.7065217391304348
recall_score =  0.7065217391304348
------------------------------
DecisionTreeClassifier
[[112  19]
 [ 27  65]]
recall =        0.7065217391304348
recall_score =  0.7065217391304348
------------------------------
LogisticRegression
[[114  17]
 [ 28  64]]
recall =        0.6956521739130435
recall_score =  0.6956521739130435
------------------------------


### 1.11
Выберите ту модель, на которой метрика recall давала лучший ответ, и рассчитайте для нее precision, используя встроенную функцию.

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
### YOUR CODE HERE ###

print('GaussianNB')
y_pred = model_gnb.predict(x_test)
print('precision_score = ', precision_score(y_test, y_pred))


GaussianNB
precision_score =  0.7647058823529411
